In [ ]:
import os
import ee
import folium

In [ ]:
ee.Initialize()

In [ ]:
import geemap
from geemap import ml
import pandas as pd
from sklearn import ensemble

In [ ]:
import ipywidgets as widgets

In [ ]:
import pprint

## Voilà !!

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:', value='Sentinel-2 Median Composite', width=200, style=style
)

bands = widgets.Dropdown(
    description='Select RGB visualization:',
    options=[
        'RED/GREEN/BLUE',
        'NIR/RED/GREEN',
        'SWIR1/NIR/GREEN',
        'SWIR2/SWIR1/NIR',
        'NIR/SWIR1/RED',
        'SWIR2/NIR/RED',
        'SWIR2/SWIR1/RED',
        'SWIR1/NIR/BLUE',
        'NIR/SWIR1/BLUE',        
        'SWIR1/NIR/RED',
        'BLUE/GREEN/RED/NIR/NIRa/SWIR1/SWIR2/Redg1/Redg2/Redg3/NDVI/gNDVI/NBR/EVI/EVI2/NDMI/NDWI/MNDWI/ARI2/ChlR/MVI/SAVI/BSI/IBI',
    ],
    # value='SWIR1/NIR/GREEN',
    value='BLUE/GREEN/RED/NIR/NIRa/SWIR1/SWIR2/Redg1/Redg2/Redg3/NDVI/gNDVI/NBR/EVI/EVI2/NDMI/NDWI/MNDWI/ARI2/ChlR/MVI/SAVI/BSI/IBI',
    style=style,
)

hbox1 = widgets.HBox([title, bands])
hbox1

In [ ]:
# bands.value

In [ ]:
# text = widgets.Text(
#     value='Madagascar',
#     placeholder='Enter a place name',
#     description='Country',
#     disabled=False,
# )
# text

# radio_buttons = widgets.RadioButtons(
#     options=[
#         'SWIR1',
#         'NIR',
#         'GREEN'],
#     value='SWIR1',
#     description='Band:'
# )
# radio_buttons

In [ ]:
# radio_buttons.value

In [ ]:
# start_year = widgets.IntSlider(
#     description='Start Date:',value=2015, min=2015, max=2022, style=style
# )
# end_year = widgets.IntSlider(
#     description='End Date:', value=2022, min=2015, max=2022,style=style
# )

import datetime

start_date = widgets.DatePicker(
    description='Start Date:', disabled=False, value=datetime.date(2015,1,1), style=style
)
end_date = widgets.DatePicker(
    description='End Date:', disabled=False, value=datetime.date.today(), style=style
)

start_month = widgets.IntSlider(
    description='Start Month:', value=1, min=1, max=12, style=style
)
end_month = widgets.IntSlider(
    description='End Month:', value=12, min=1, max=12, style=style
)

hbox2 = widgets.HBox([start_date, end_date, start_month, end_month])
hbox2

In [ ]:
# start_date.value.strftime("%Y-%m-%d")

In [ ]:
# start_date.value

In [ ]:
create_composite = widgets.Button(
    description='Create Composite',
    button_style='primary',
    tooltip='Click to create median composite',
    style=style,
)

output = widgets.Output()

hbox3 = widgets.HBox([create_composite])
hbox3

In [ ]:
def submit_clicked(b):
    with output:
        output.clear_output()
         
        if start_date.value > end_date.value:
            print('The end year must be great than the start year!')
            return
        if start_month.value > end_month.value:
            print('The end month must be great than the start month!')
            return
        if start_date.value == end_date.value:
            add_progress_bar = False
        else:
            add_progress_bar = True
            
        iDate = start_date.value.strftime("%Y-%m-%d")
        fDate = end_date.value.strftime("%Y-%m-%d")
       
        
        # start_date = str(start_month.value).zfill(2) + '-01'
        # end_date = str(end_month.value).zfill(2) + '-30'
        
        print('Computing...')
        
        coll = genCollS2(
            geom=Map.user_roi,
            # label=title.value,
            iDate=iDate,
            fDate=fDate,
            # start_date=start_date,
            # end_date=end_date,
            # bands=bands.value.split('/'),
            # font_color=font_color.value,
            # frames_per_second=speed.value,
            # font_size=font_size.value,
            # add_progress_bar=add_progress_bar,
            # progress_bar_color=progress_bar_color.value,
            # download=True,
            # apply_fmask=cloud.value,
            cpp=25
        )
        aoi=Map.user_roi
        Map.addLayer(
            coll.select(["MVI"]).median().clip(aoi),
            {
                "min": -0.35,
                "max": 5,
                "palette": [
                    "04fffe",
                    "CE7E45",
                    "DF923D",
                    "F1B555",
                    "FCD163",
                    "99B718",
                    "74A901",
                    "66A000",
                    "529400",
                    "3E8601",
                    "207401",
                    "056201",
                    "004C00",
                    "023B01",
                    "012E01",
                    "011D01",
                    "011301",
                ],
            },
            "MVI",
        )
        Map.addLayer(coll.select(['SWIR1','NIR','GREEN']).median().clip(aoi), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'Sentinel-2 Coll')
        
create_composite.on_click(submit_clicked)

In [ ]:
output

In [ ]:
Map = geemap.Map(center=(-13.5, 48.4), zoom=8)
Map.add_basemap('HYBRID')
Map

In [ ]:
# Prepare Sentinel-2 
#
def prepareS2(image):
    bandList = ['B2', 'B3', 'B4', 'B5','B6','B7', 'B8', 'B8A','B11', 'B12', 'QA60']
    nameList = ['BLUE','GREEN','RED', 'Redg1', 'Redg2', 'Redg3','NIR','NIRa','SWIR1','SWIR2','QA60']
    scaling = 10000
    # scaling = 1
    scaled = ee.Image(image).select(bandList).rename(nameList).divide(ee.Image.constant(scaling))
    return ee.Image(image).addBands(scaled)

In [ ]:
# Calculate spectral indices for all bands in collection
# @param {ee.ImageCollection} collection Landsat image collection
# @returns {ee.ImageCollection} Landsat image with spectral indices
#
def doIndS2(image):
    def func_gev(image):
        NDVI = calcNDVI(image)
        gNDVI = addGNDVI(image)
        NBR = calcNBR(image)
        EVI = calcEVI(image)
        EVI2 = calcEVI2(image)
        NDMI = addNDMI(image)
        NDWI = addNDWI(image)
        MNDWI = addMNDWI(image)
        MVI = addMVI(image)
        ARI2 = addARI2(image)
        ChlR = addChlR(image)
        BSI = addBSI(image)
        IBI = addIBI(image)
        SAVI = addSAVI(image)        
        # TC = tcTrans(image)
        # NDFI function requires surface reflectance bands only
        # BANDS = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']
        # NDFI = calcNDFI(image.select(BANDS))
        return image.addBands([NDVI, gNDVI, NBR, EVI, EVI2, NDMI, NDWI, MNDWI, ARI2, ChlR, MVI, BSI, IBI, SAVI]) #
    return image.map(func_gev)

In [ ]:
# Function to cloud mask from the QA60 (Sentinel-2) and from the pixel_qa band of Landsat 8 SR data
def maskS2clouds(image):
    QA60 = image.select(['QA60'])
    clouds = QA60.bitwiseAnd(1 << 10).Or(QA60.bitwiseAnd(1 << 11))
    return image.updateMask(clouds.Not())

In [ ]:
# Generate filtered (date and cloud masks) collection of Sentinel-2 images
 # @param {ee.Image} geom Geometry used to filter the collection
 # @param {String} startDate Initial date to filter the collection
 # @param {String} endDate Final date to filter the collection
 # @param {number} cpp stands for 'Cloudy Pixel Percentage'
 # @returns {ee.ImageCollection} Filtered Sentinel-2 collection
#
def genCollS2(geom, iDate, fDate, cpp):
    collect_S2 = (ee.ImageCollection('COPERNICUS/S2')
                  .filterDate(iDate, fDate)
                  .filterBounds(geom)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cpp))
                  .map(maskS2clouds))
    prep_S2 = collect_S2.map(prepareS2)
    indices = doIndS2(prep_S2).select(['BLUE','GREEN','RED','NIR', 'NIRa','SWIR1','SWIR2',
                                       'Redg1', 'Redg2', 'Redg3','NDVI','gNDVI','NBR','EVI','EVI2',
                                       'NDMI','NDWI','MNDWI','ARI2','ChlR','MVI','SAVI','BSI','IBI']) 
    return ee.ImageCollection(indices)    

In [ ]:
# 1. Calculate/addBands NDVI for an image
# @param {ee.Image} image,  e.g., Landsat or Sentinel-2 image with NIR and RED bands
# @returns {ee.Image} NDVI image
#
def calcNDVI(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR','RED']).rename('NDVI')
    return ndvi

def addNDVI(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR','RED']).float()
    return image.addBands(ndvi).rename('NDVI')

In [ ]:
# 2. Calculate 'Green' NDVI for an image
# @param {ee.Image} image,  e.g., Landsat image with NIR and RED bands
# @returns {ee.Image} GNDVI image
#
def addGNDVI(image):
    gndvi = ee.Image(image).normalizedDifference(['NIR','GREEN']).float()
    return image.addBands(gndvi.rename('gNDVI'))

In [ ]:
# 3. Calculate Standard Deviation of NDVI values over a image collection
#
def addNDVIsd(image):
    ndviStd = ee.ImageCollection(image).select('NDVI').reduce(ee.Reducer.stdDev())
    # reducer = ee.Reducer.stdDev()
    # ndviStd = ndvi.reduce(ee.Reducer.median())
    # return image.addBands(ndviStd.rename('ndviSD'))
    return ndviStd.rename('ndviSD')

In [ ]:
# 4. Calculate NBR for an image
# @param {ee.Image} image  Landsat image with NIR and SWIR2 bands
# @returns {ee.Image} NBR image
#
def calcNBR(image):
    nbr = ee.Image(image).normalizedDifference(['NIR', 'SWIR2']).rename('NBR')
    return nbr

In [ ]:
# 5. Calculate EVI for an image
# @param {ee.Image} image Landsat image with NIR, RED, and BLUE bands
# @returns {ee.Image} EVI transform
#
def calcEVI(image):
    evi = ee.Image(image).expression(
          'float(2.5*(((B4) - (B3)) / ((B4) + (6 * (B3)) - (7.5 * (B1)) + 1)))',
          {
              'B4': ee.Image(image).select(['NIR']),
              'B3': ee.Image(image).select(['RED']),
              'B1': ee.Image(image).select(['BLUE'])
          }).rename('EVI')
    return evi

In [ ]:
# 6. Calculate EVI2
#
def calcEVI2(image):
    evi2 = ee.Image(image).expression(
        'float(2.5*(((B4) - (B3)) / ((B4) + (2.4 * (B3)) + 1)))',
        {
            'B4': image.select('NIR'),
            'B3': image.select('RED')
        }).rename('EVI2')
    return evi2

In [ ]:
# 7. Calculate/addBands: NDMI
#
def addNDMI(image):
    ndmi = ee.Image(image).normalizedDifference(['NIR', 'SWIR1']).float().rename('NDMI')
    return image.addBands(ndmi)

In [ ]:
# 8. Calculate/addBands: NDWI
#
def addNDWI(image):
    ndwi = ee.Image(image).normalizedDifference(['GREEN', 'NIR']).float().rename('NDWI')
    return image.addBands(ndwi)

In [ ]:
# 9. Calculate and addBands: MNDWI
#
def calcMNDWI(image):
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).float().rename('MNDWI')
    return mndwi

def addMNDWI(image):
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).float().rename('MNDWI')
    return image.addBands(mndwi)

In [ ]:
# 10. Calc/addBands: MVI (Mangrove Vegetation Index, see SEPAL)
#
def addMVI(image):
    mvi = ee.Image(image).expression(
        '(1.0 * (NIR - GREEN) / abs(SWIR1 - GREEN))',
        {
            'GREEN': image.select('GREEN'),
            'NIR': image.select('NIR'),
            'SWIR1': image.select('SWIR1')
        }).float().rename('MVI')
    return image.addBands(mvi)

In [ ]:
# 11. Calculate/addBands: ARI2 [Sentinel-2 only]
#
def addARI2(image):
    ari2 = ee.Image(image).expression(
        '((1.0 / GREEN) - (1.0 / Redg1)) * Redg3',
        {
            'GREEN': image.select(['GREEN']),
            'Redg1': image.select(['Redg1']),  # 'B5'
            'Redg3': image.select(['Redg3'])   # 'B7'
        }).float().rename('ARI2')
    return image.addBands(ari2.rename('ARI2'))

In [ ]:
# 12. Calculate/addBands: ChlR [Sentinel-2 only]
#
def addChlR(image):
    chlRedg = ee.Image(image).expression(
        'pow((Redg3 / Redg1), -1.0)',
        {
            'Redg1': image.select(['Redg1']),
            'Redg3': image.select(['Redg3'])
        }).float().rename('ChlR')
    return image.addBands(chlRedg)

In [ ]:
# 13.Calculate/addBands: SAVI
#
def addSAVI(image):
    savi = ee.Image(image).expression(
        '(NIR - RED)/ (NIR + RED + 0.428) * (1.0 + 0.428)',
        {
            'NIR': image.select('NIR'),
            'RED': image.select('RED')
        }).float().rename('SAVI')
    return image.addBands(savi)

In [ ]:
# 14. Calculate/addBands: BSI (bare soil index)
#
def addBSI(image):
    bsi = ee.Image(image).expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))',
        {
            'BLUE': image.select('BLUE'),
            'RED': image.select('RED'),
            'NIR': image.select('NIR'),
            'SWIR1': image.select('SWIR1')
        }).float().rename('BSI')
    return image.addBands(bsi)

In [ ]:
# 15. Calculate/addBands: IBI (index-based built-up index)
# 
def addIBI(image):
    ibiA = ee.Image(image).expression('2 * SWIR1 / (SWIR1 + NIR)',
                                      {
                                          'SWIR1': image.select('SWIR1'), # S2: 'B11'
                                          'NIR': image.select('NIR'), # S2: 'B8'
                                      }).rename('IBI_A')
    ibiB = ee.Image(image).expression('(NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))',
                                      {
                                          'NIR': image.select('NIR'),
                                          'RED': image.select('RED'),
                                          'GREEN': image.select('GREEN'),
                                          'SWIR1': image.select('SWIR1')
                                      }).rename('IBI_B')
    ibiAB = ibiA.addBands(ibiB)
    ibi = ibiAB.normalizedDifference(['IBI_A', 'IBI_B'])
    return image.addBands(ibi.rename('IBI'))

In [ ]:
 # 16. Calculate NDFI using endmembers from Souza et al., 2005
 # @param {ee.Image} Surface reflectance image with 6 bands (i.e. not thermal)
 # @returns {ee.Image} NDFI transform
 #
def calcNDFI(image):
  # Do spectral unmixing #
    gv = [.0500, .0900, .0400, .6100, .3000, .1000]
    shade = [0, 0, 0, 0, 0, 0]
    npv = [.1400, .1700, .2200, .3000, .5500, .3000]
    soil = [.2000, .3000, .3400, .5800, .6000, .5800]
    cloud = [.9000, .9600, .8000, .7800, .7200, .6500]
    cf = .1 # Not parameterized
    cfThreshold = ee.Image.constant(cf)
    unmixImage = (ee.Image(image).unmix([gv, shade, npv, soil, cloud], True,True)
                  .rename(['band_0', 'band_1', 'band_2','band_3','band_4']))
    newImage = ee.Image(image).addBands(unmixImage)
    mask = unmixImage.select('band_4').lt(cfThreshold) # Check that this is the right 'image'; previously 'Image'
    ndfi = ee.Image(unmixImage).expression(
        '((GV / (1 - SHADE)) - (NPV + SOIL)) / ((GV / (1 - SHADE)) + NPV + SOIL)', 
        {
            'GV': ee.Image(unmixImage).select('band_0'),
            'SHADE': ee.Image(unmixImage).select('band_1'),
            'NPV': ee.Image(unmixImage).select('band_2'),
            'SOIL': ee.Image(unmixImage).select('band_3')
    })
    return ee.Image(newImage) \
        .addBands(ee.Image(ndfi).rename(['NDFI'])) \
        .select(['band_0','band_1','band_2','band_3','NDFI']) \
        .rename(['GV','Shade','NPV','Soil','NDFI']) \
        .updateMask(mask)

In [ ]:
# 17. Tassel Cap coefficients from Crist 1985
# @param {ee.Image} image, Landsat image with BLUE, GREEN, RED, NIR, SWIR1, and SWIR2
# @returns {ee.Image} 3-band image with Brightness, Greenness, and Wetness
#
def tcTrans(image):
    # Calculate tasseled cap transformation
    brightness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': 0.2043,
            'L2': image.select('GREEN'),
            'B2': 0.4158,
            'L3': image.select('RED'),
            'B3': 0.5524,
            'L4': image.select('NIR'),
            'B4': 0.5741,
            'L5': image.select('SWIR1'),
            'B5': 0.3124,
            'L6': image.select('SWIR2'),
            'B6': 0.2303
        })
    greenness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': -0.1603,
            'L2': image.select('GREEN'),
            'B2': -0.2819,
            'L3': image.select('RED'),
            'B3': -0.4934,
            'L4': image.select('NIR'),
            'B4': 0.7940,
            'L5': image.select('SWIR1'),
            'B5': -0.0002,
            'L6': image.select('SWIR2'),
            'B6': -0.1446
        })
    wetness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': 0.0315,
            'L2': image.select('GREEN'),
            'B2': 0.2021,
            'L3': image.select('RED'),
            'B3': 0.3102,
            'L4': image.select('NIR'),
            'B4': 0.1594,
            'L5': image.select('SWIR1'),
            'B5': -0.6806,
            'L6': image.select('SWIR2'),
            'B6': -0.6109
        })

    bright =  ee.Image(brightness).rename('BRIGHTNESS')
    green = ee.Image(greenness).rename('GREENNESS')
    wet = ee.Image(wetness).rename('WETNESS')

    tasseledCap = ee.Image([bright, green, wet])
    return tasseledCap